# 爬取網站 : [IT邦幫忙](https://ithelp.ithome.com.tw/)
## 抓取內容 : 文章的標題、標籤、詢問日期、瀏覽次數、內文
### **P.S.1 因為開檔的mode是使用'a'附加資料的方式寫入，所以使用前請先確認JSONcrawler.json檔案不存在，否則資料會有重疊**
#### P.S.2 標籤及內文為了避免跟JSON檔案格式衝突，因此有使用REGEX替換特定符號，替換前及替換後如下
#### ( (" -> ')　, 　(\ -> BACKSLASH)　,　(換行符號(\n) -> EOF)　,　(Tab(\t) ->     (四個空白)) )

# 函式

In [1]:
import urllib.request as req #引入urllib模組中函式名稱為request的函式為req
import bs4 #引入bs4模組
import re #引入re模組
#定義函式getData，用以取得每一頁所有標題、標籤及內文。參數為網址
def getData(url,count,pages):
    #建立一個Request物件，附加Request Headers的資訊
    request = req.Request(url,headers = {
        "User-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.0.0 Safari/537.36"
    })
    
    #打開request
    with req.urlopen(request) as response:
        data = response.read().decode("utf-8")

    root = bs4.BeautifulSoup(data,"html.parser") #讓BeautifulSoup解析HTML格式文件
    all_titles = root.find_all("h3",class_="qa-list__title") #以列表形式找出所有class_=title的h3標籤


    #開啟JSONcrawler.json並用a模式保留原本資料並將新的資料寫入
    with open("JSONcrawler.json",mode = "a",encoding = "utf-8") as ftext:
        #此迴圈會找出該頁所有標題、標籤、詢問時間、瀏覽次數及內文
        for title in all_titles:
            title.a.string = re.sub(r"\"","'",title.a.string) #利用regex將 " 改成 ' ，以免語法錯誤
            title.a.string = re.sub(r"\\"," BACKSLASH ",title.a.string) #利用regex將 \ 改成 BACKLASH ，以免語法錯誤
            ftext.write("\n   {\n") #寫入與JSON檔的檔案格式有關的detail
            ftext.write("   \"標題\":"+"\""+title.a.string+"\","+"\n") #將內文的標題寫入JSON檔裡
            print("標題 : "+title.a.string)
            
            ftext.write("   \"標籤\":[") #寫入與JSON檔的檔案格式有關的detail

            url2 = title.a["href"] #將內文的網址存到url2裡

            #建立一個Request物件，附加Request Headers的資訊
            request2 = req.Request(url2,headers = {
                "User-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.0.0 Safari/537.36"
            })

            #打開request2
            with req.urlopen(request2) as response2:
                data2 = response2.read().decode("utf-8")
            
            root2 = bs4.BeautifulSoup(data2,"html.parser") #讓BeautifulSoup解析HTML格式文件
            contents = root2.find("div",class_="markdown__style").text #找出內文
            tags = root2.find("div",class_="qa-header__tagGroup") #以列表形式找出所有 class_=qa-header__tagGroup 的 div 標籤
            time_and_view = root2.find("div",class_="qa-header__info").text.split() #以列表形式找出所有 class_=qa-header__info 的 div 標籤
            
            #利用regex將會影響JSON的符號做替換
            contents = re.sub(r"\"","'",contents)
            contents = re.sub(r"\\"," BACKSLASH ",contents)
            contents = re.sub(r"\n","EOL",contents)
            contents = re.sub(r"\t","    ",contents)

            set_of_tags = set(tags.text.split("\n")) #利用集合將重複出現的標籤強制讓它出現一次
            #將集合內的資料重新裝回list內
            list_of_tags = []
            for i in set_of_tags:
                list_of_tags.append(i)
            
            #將標籤寫進JSON檔
            for tag in list_of_tags[1:]:
                tag = re.sub(r"\\"," BACKSLASH ",tag)
                if (tag!=list_of_tags[-1]):
                    ftext.write("\"" + tag + "\",")
                else:
                    ftext.write("\"" + tag + "\"],\n")
            
            print("標籤 :",end = " ")
            for tag in list_of_tags[1:]:
                print(tag,end = " ")
            
            ftext.write("   \"詢問時間\":\""+time_and_view[1]+"\",\n") #將詢問時間寫入JSON檔
            print("\n詢問時間 : "+time_and_view[1])
            ftext.write("   \"瀏覽次數\":\""+time_and_view[-2]+"\",\n") #將瀏覽次數寫入JSON檔
            print("瀏覽次數 : "+time_and_view[-2])

            #將內文寫入JSON檔
            if ( (count==pages-1) and (title.a.string == all_titles[-1].a.string)):
                ftext.write("   \"內文\":\""+contents+"\"\n") #將內文寫入content.txt裡
                ftext.write("   }]\n}")
                print("內文 : "+contents)
            else:
                ftext.write("   \"內文\":\""+contents+"\"\n")
                ftext.write("   },\n")
                print("內文 : "+contents+"\n\n")


    #抓取下一頁的連結
    nextLink = root.find("a",string="下一頁") #找到內文是 下一頁 的標籤a
    return nextLink["href"] #回傳下一頁的網址


# 主程式(要求使用者輸入想抓取多少頁數的資料)

## 此處以抓取50頁資料為例(約3個月份的資料量)

In [2]:
#寫入與JSON檔的檔案格式有關的detail
with open("JSONcrawler.json",mode = "a",encoding = "utf-8") as ftext:
    ftext.write("{\n")
    ftext.write("   \"Article\":[")
pageURL = "https://ithelp.ithome.com.tw/" #將起始網頁的網址存到pageURL裡
count = 0 #count用來表示想抓取幾頁
pages = int(input("Please enter how many pages you want to crawl:"))
#重複抓取每頁資料
while count<pages:
    pageURL = getData(pageURL,count,pages) #將 return 回來的網址覆蓋到pageURL上，以便重複利用
    count += 1 #執行完一次後增加1，以確保能抓到預期的頁數

Please enter how many pages you want to crawl:50
標題 : Google等其他搜尋引擎爬蟲爬取的網頁原始碼是?
標籤 : 網頁 原始碼 chrome 爬蟲 google 
詢問時間 : 2022-11-16
瀏覽次數 : 3
內文 : EOL拿大家最常用的chrome瀏覽器來看，網頁查看原始碼方式有二種EOL1.鍵盤Ctrl+U查看源代碼：就是網頁服務器發送到瀏覽器的最原始代碼EOL2.鍵盤F12檢查元素：看到的就是最終的html代碼。即：源代碼 + 網頁js渲染EOL有些網頁是Javascript渲染的技術建置,所以會有Ctrl+U查不到文字,而F12查的到內文EOL因此想問大家，Google爬蟲的話,是爬取Ctrl+U還是F12的原始碼?EOL


標題 : 用子母畫面顯示文字
標籤 : #canvas #前端 #html js javascript 
詢問時間 : 2022-11-16
瀏覽次數 : 138
內文 : EOL我想要用JS及時生成HTML Video來顯示文字EOL這樣可以用子母畫面顯示一些會即時更新的文字資訊EOL不過不太確定怎麼把文字或Canvas及時轉成影片EOL求各位大神幫忙EOL示意圖EOLEOL


標題 : docker_host推版時回覆error during connect: Post......exited with exit status 255
標籤 : docker_host ssh docker ssh key docker-compose 
詢問時間 : 2022-11-16
瀏覽次數 : 88
內文 : EOL大家好,遇到的問題是...EOL原本在本機端更新主機版本都使用如下指令 :EOL'SPRING_ENV=qa DOCKER_HOST='ssh://XXX@XX.XX.XX.XX' docker-compose -f docker-compose-tw.yml up -d --build'EOL但是後來用一樣的指令會被回覆EOLEOL上網查了資訊,有人說是Key沒有加EOL不過檢查了key,確定有加了EOL也有人說是因為主機和client端的SSH版本不同EOL也有針對SSH新舊版本對Key的加密解法加了相容的演算法EOL但仍是一樣的訊息EOL想問問有沒有人遇

標籤 : 編輯軟體 編輯器 文字編輯軟體 文字編輯器 
詢問時間 : 2022-11-15
瀏覽次數 : 451
內文 : EOL在PHP程式中使用了一段其他機構提供的邏輯判斷EOL但是光一段邏輯判斷就有37萬個字元(後續還有好幾段要引入)EOL我的作業系統是WindowsEOL平常在用的編輯軟體是MadEditEOL但是使用MadEdit編輯這個大量內容的程式時會有很明顯的延遲感EOL在編輯時也相當不順暢EOL後來改用Windows內建的記事本編輯時就不會有延遲的問題EOL但是就缺乏MadEdit的顏色標記等功能EOL請問有沒有其他推薦的純文字編輯軟體EOL能夠提供程式標記等相關功能EOL又不會因為程式內容過多而有所延遲EOL註：有想過將邏輯判斷以include的方式獨立到其他支程式，但是這些邏輯判斷有時也需要進行編輯。所以就只是將明顯的編輯延遲問題減緩一點點而已。EOL看錯位數了，修正一下：一段邏輯判斷就有3.7萬個字元EOL


標題 : JQUERY DropDownList 請教
標籤 : jquery 
詢問時間 : 2022-11-15
瀏覽次數 : 240
內文 : EOL我想請問一下如果今天我有個下拉選單EOLhttp://jsfiddle.net/jsmnwuoh/1/EOL如果說我有點下拉選單然後點個重置的按鈕的話EOL下拉選單就會變回請選擇這個選項EOL請問該如何實現呢?EOL我目前只有做到取到他的字但沒辦法畫面整個修改EOL


標題 : 關於POE供電接非POE設備
標籤 : 轉接 poe 
詢問時間 : 2022-11-15
瀏覽次數 : 342
內文 : EOL因目前有位置要新增電腦，但是當初並沒有拉線至現場，只有當時廠商安裝監視器的POE HUB，EOL我看了一下該設備是沒有掛廠牌的，所以我挺擔心他的品質，不敢直接接主機，上網查了一下好像大多都是利用4578走供電(模式B)，EOL那我是否可以透過不使用4578來避免POE HUB可能沒有802.3AT/AF或未來故障的時候，不會沖壞我的設備，EOL或著前端加一台HUB防衝，更甚至去買一個網路與電源分接的小東西來做處理?EOL正常廠商要壓POE成本的模式下，應該會是走模式B比較節省成本?EOL那假如今天運氣很差真的是走模式A的話，又是持續供電無規範的，就只能買個分接小東西了?EOL


標題 : 無法讀取隨身碟
標籤 : 隨身碟 
詢問時間 : 2022-11-14
瀏覽次數 : 343
內文 : EOL請教大家EOL在win10作業系統中, 插入隨身碟後, 無法讀取隨身碟的內容, 有作下列的測試EOL1.同一台電腦使用別的作業系統可以讀取到隨身碟 --> 所以代表usb及線都正常EOL2.有發現隨身碟的屬性被改成 readonly, 透過指令方式將其修正 (attributes disk clear readonly)EOL但在原來的作業系統上, 還是顯示: 存取被拒, 但我又沒辦法直接在安全性來套用權限設定, 在套用設定的過程,無法完成套用, 有指令可以套用權限嗎?EOL註: 非常有可能在作業系統中有安裝或者植入usb保護工具, 但要如何找到該常駐軟體呢EOLEOL無法改權限EOLEOL謝謝EOL


標題 : 不同字數部落格文章的Hashtag標籤上限最好幾個?
標籤 : tag 部落格 hashtag 標籤 blog 
詢問時間 : 2022-11-14
瀏覽次數 : 201
內文 : EOL如標題，請問有人知道,部落格文章大概有多少字就設定一個Hashtag標籤(舉例：#SEO)會比較好?EOL補充：Hashtag標籤跟超連結不一樣噢EOL


標題 : windows server 2016 使用造字程式 點文字服務連結 會顯示造字程式已經停止運作 使用微軟注音
標籤 : windows server 軟體 
詢問時間 : 2022-11-14
瀏覽次數 : 178
內文 : EOLwindows server 2016 使用造字程式 點文字服務連結 會顯示造字程式已經停止運作 使用微軟注音EOLEOLEOL


標題 : 關於 MySQL 資料庫 執行 revoke 撤銷權限問題
標籤 : mysql sql revoke 資料庫 
詢問時間 : 2022-11-14
瀏覽次數 : 213
內文 : EOL您好， MySQL 資料庫版本是 8.0.23 版。EOL遇到的問題是，當我對某一個使用者在資料庫中的特定資料表執行 revoke 指令回收權限的時候，會發生如下圖般的錯誤訊息。EOLEOL但是如果我不指定 資料庫，執行 Revoke 回收 Update 權限，卻又可以正常執行。EOLEOL如果我對某一個資料庫，但是不指定資料表，執

標籤 : asp.net c# 
詢問時間 : 2022-11-13
瀏覽次數 : 342
內文 : EOL各位好:EOL我在後端有作一些SQL的查詢，想把查詢的結果(LIST)傳到前端javascriptEOL後端test.csEOL//code 省略，查詢SQL資料，存入datatable dr中....EOLList<string>name=new List<string>EOL name = dt.Select().Select(dr => dr['name'].ToString().Trim()).ToList();EOLEOL前端test.aspxEOL  <script type='text/javascript'>EOL     EOL        var myFilters = [EOL            EOL                {EOL                    id: '1',EOL                        label: '1',EOL                            type: 'string'EOL                },EOL             EOL        ];EOL         EOL        var musicList ='<%= this.name %>';//這裡會錯誤，我都抓不到值，會讓整個js出不來EOL        for (var i = 0; i < musicList.length; i++) {EOL          EOL            myFilters.push({EOL                id: musicList[i],EOL                label: musicList[i],EOL                type: 'string'EOL            },EOL            );EOL        }EOL        </script>EOLEOL謝謝EOL


標題 : StringBuilder相關問題請教
標籤 : c# 
詢問時間 : 2022-11-13
瀏覽次數 : 259
內文 : EOL我想請問一下關

標籤 : access 
詢問時間 : 2022-11-11
瀏覽次數 : 265
內文 : EOL用Access設計報表，於報表檢視時格式正常，但嘗試列印或匯出pdf時，卻發現文件第一頁只顯示部份報表內容，餘下的報表內容卻移至第二頁，請問出了什麼問題? 謝謝。EOL


標題 : excel 2007是不是沒有支援 xlookup 函式啊?
標籤 : excel vlookup/xlookup xlookup 
詢問時間 : 2022-11-11
瀏覽次數 : 357
內文 : EOLexcel 2007是不是沒有支援 xlookup 函式啊?EOL在sheets成功但是excel裡不行?EOL我該如何讓他支援呢?EOL


標題 : DNS lookup問題 (DC nslookup unknown, 但網域成員nslookup成功?)
標籤 : dns 
詢問時間 : 2022-11-11
瀏覽次數 : 293
內文 : EOL小弟goolge過但問題好像不太一樣所以暫時找不到解決方法所以來問問大神們EOL情況是這樣, 小弟公司的網域有三台DC (1.1, 1.2, 1.3)EOL主要DC是1.2, 5大角色都在1.2EOL而近來因為1.1的DNS顯示出有問題, 加上本來1.1已經感覺OS有問題的樣子EOL咨詢了主管後就直接降級然後把它退了役, 然後打算再裝新一台server升級DC取代原本的1.1EOL但都卡在了升級DC時出error(大概是複寫失敗, 網絡路徑找不到之類)EOL現在回想好像降級舊DC的程序不太規範, 可能影響了現役DC的樣子...(這我是猜測)EOL於是就找問題, 後來發覺了nslookup有問題所以猜測是這樣影響了DC升級的程序以致失敗)EOL但奇怪點是現役DC nslookup是unknownEOLEOL但其他成員nslookup倒是成功EOLEOL這我就覺得很奇怪, 查了A record, PTR record跟reserve zone也有加相應的DC (DNS)EOL所以不明白是那裡出了問題EOL小弟還是新手, 想問問大神們我該往那方面做troubleshoot.EOL先行謝過EOL


標題 : Docker使用alpine時，無法執行程式
標籤 : docker 
詢問時間 : 2022-11-10
瀏覽次數 : 478
內文 

...

標籤 : fortigate 80mhz 5g 頻道選擇 
詢問時間 : 2022-07-20
瀏覽次數 : 1119
內文 : EOLEOL最近剛看FortiOS7.2的介紹,提到的wifi設定介面(如上圖), 讓我對我這兒的FortiOS6.4.8的選項產生了疑問.EOL40MHz 他列表就把佔用的兩頻道另一個弄消失.EOLEOL80MHz 卻沒有消失更多,而是全部列出來?EOLEOL讓我越來越迷糊,想請教:EOLEOLEOL如果我想跑80MHz的UNII-3, 我應該149-161 都勾選嗎? 還是只要勾選其中一個如 149 即可?EOLEOLEOL我這裡有兩顆相鄰的fortiAP, FortiAP Profile都選 80MHz (channel 36-161全選), 但是從fortigate上看到, 他一個跑channel 153, 另一個跑channel 161.EOL這樣的選擇豈不是互相干擾?EOLEOLEOL有人跟我說,Forti的channel width 都是向下相容, 所以選擇 80MHz, 其實也支援40MHz、 20MHz, 是這樣嗎?EOL如果是, 這樣就解釋了我的問題2,因為她不是真的跑80Mhz?EOL有辦法從fortigate上看到client是跑20/40/80MHz哪種嗎?EOLPs. 頻道段該怎麼選我是看到下圖產生是否要全選的疑問, 我應該全選? 或者下圖打X不要選, 選擇 36, 64, 100, 116, 132, 149.  我這裡的FortiAP profile 有40MHz跟80MHz兩種.EOLEOLEOLEOL


標題 : 關於WIN  10 內建 L2TP  VPN  問題
標籤 : 防火牆 網路架構 vpn win10vpn l2tp 
詢問時間 : 2022-07-20
瀏覽次數 : 1720
內文 : EOL我有兩間公司 一間 稱為 A公司  (IP:172.16.開頭)、另一間為 B公司 (IP:172.17.開頭)，兩間距離很遠EOL最近A公司 新購防火牆，我利用這台防火牆 設定了VPN，使用L2TP連線方式EOL然後我在家裡WIN10 電腦 成功設定好內建的VPN，可以順利連到 A公司EOL但目前遇到一個問題如下:EOL在我家電腦的VPN連線狀態下，會使用預設的VPN 閘道，如下圖EOLEOL但這

標籤 : line mikrotik 
詢問時間 : 2022-07-19
瀏覽次數 : 1181
內文 : EOL上次提出的問題EOLhttps://ithelp.ithome.com.tw/questions/10208719EOL感謝各位先進的指引，後來找到資料參考也順利解決了。EOL但這兩天卻又出現一個怪現象，就是LINE無法傳圖片、檔案。EOL在月初之前，這個功能確定是可正常使用的，Mikrotik (RouterOS 6.49.6)的設定未作任何更動。但只要透過Mikrotik，就無法傳送圖片、檔案，其他文字、語音、視訊皆可正常使用。而網路移除Mikrotik後，LINE就能正常傳送圖片、檔案。EOL目前Mikrotik防火牆最後一條是設定阻止所有進入的封包，在這條的前面則設定開放LINE網域的L7-FilterEOL^.+(.line.).*$EOL及LINE IP網段EOL147.92.128.0/17EOL203.104.128.0/20EOL以及LINE租用的Google LLC、AWS-EC2主機EOL請問還需要開放什麼特別的設定嗎？EOL


標題 : Node.js的開發工具
標籤 : node.js 
詢問時間 : 2022-07-19
瀏覽次數 : 651
內文 : EOL我本身從事.net開發，所以使用的開發工具是Visual Studio，而我過去曾有短暫學過Node.js，學習的教材都是以vs code做為教學範例。而近期工作上需要學習Node.js，同事也都用vs code來開發。但我想請問各位是否有使用過Visual Studio開發node.js的經驗? 因為它也有相關的開發工具，所以用起來會比較強大有力嗎? 因為我知道vs code也能寫C#, 但聽說很難用, 不如用VS。想說用VS寫node.js是否有任何優點或好處? 謝謝EOL


標題 : 私心想請問各位前輩成為CTO具備什麼能力
標籤 : #it管理 
詢問時間 : 2022-07-19
瀏覽次數 : 3035
內文 : EOL會問這個問題是因為在IT界當工程師打滾幾年到管理職，每年技術推陳出新，但年紀越大記憶越差，除了，除了這幾年累積的經驗和處理突發問題及帶團隊能力外，想請問各位前輩怎樣的一條路才是最為建議的，因為邏輯、分析及整合性是我所擅長的也最終確定CTO方

標籤 : vcenter 7 vmware vcenter vcenter 
詢問時間 : 2022-07-18
瀏覽次數 : 485
內文 : EOL求幫忙EOL有人有遇過vCenter編輯設定>硬碟無法調整的狀況嗎EOL硬碟的部分有做HAEOL目前都在ESXi修改EOLvSphere Client 7.0.2.00500 版EOLEOLEOLEOL新增虛擬機的時候出現'無法連線到Profile-Driven Storage Server'EOLEOL之前有因為憑證問題做過EOL/usr/lib/vmware-vmca/bin/certificate-managerEOL不知道會不會有影響EOL


標題 : pyautogui編譯成exe圖片無法定位
標籤 : @python @pyautogui @pyinstaller 
詢問時間 : 2022-07-18
瀏覽次數 : 366
內文 : EOLpython 3.10.4版EOL使用pyautogui圖片定位去模擬滑鼠去按某個功能EOL在還沒編譯console中執行,所有功能均正常EOL用pyinstaller跟nuitka編譯成exe後EOL圖片辨識地位都不能動了,定位也沒顯示None,沒錯誤EOLpyautogui.locateCenterOnScreen() 完全沒反應EOLgrayscale跟confidence都有排列組合使用EOLpython版本也用pyenv去嘗試3.7 3.6版使用EOL都是console中正常,exe後沒反應EOL請問大大們有沒有人有解?EOL


標題 : 未知錯誤
標籤 : html php 
詢問時間 : 2022-07-18
瀏覽次數 : 686
內文 : EOL小弟我照著以前的方法寫php，但是出現了之前沒出現過的錯誤，請各位大大解惑。EOL<?phpEOLrequire_once('db/db.php');EOL?>EOLEOL<?phpEOL$sql = 'select Month from a4';EOL$result = $project->query($sql);EOL$row_cnt = $result->rowCount();EOL$row = $result->fetchALL(PDO::FETCH_ASSOC);EOLecho ' <font co

標籤 : directions api google directions api google map 
詢問時間 : 2022-07-16
瀏覽次數 : 498
內文 : EOL各位大大好，目前有個功能需求是想要利用手機,利用開始位置及結束位置取得路線資料,並在行車時提示使用者下一步的路線。EOL目前已經可以利用Google directions api 取得每個轉折點的路線敘述了。EOL例如:台北小巨蛋 -> 台北101EOL所撈到的路線敘述為:EOL1.往北走敦化北路朝南京東路四段前進EOL2.在第 1 個十字路口向右轉，走南京東路四段EOL....EOL10.於市府路迴轉道處迴轉 目的地在右邊EOL目前的疑問是該如何提示使用者 下一步該如何走呢 (該取哪一個路線敘述)EOL甚至是使用者若走錯路 及時需要變更路線等等...EOL我目前的做法是:EOL根據手機位置 移動距離EOL每X公尺發一次API 取最新路線資料 (這時候開始位置更改成當下位置，結束位置不變)，EOL並拿取第一個敘述來提示使用者EOL但是這個方法肯定會存在許多問題...EOL再請各位大神給小弟一個思路 謝謝！EOL


標題 : Uart接收資料問題
標籤 : stm32 uart 
詢問時間 : 2022-07-16
瀏覽次數 : 450
內文 : EOL我想透過UART持續接收感測器端的資料，長度固定，資料結尾為FF FF FF FF；目前我的作法，僅透過資料串列是否有連續四個FF來判斷資料結尾。但這種作法並不完善，請問題是否能給我一些修改的建議方向?如下是我目前主要的程式碼。EOLvoid HAL_UART_RxCpltCallback(UART_HandleTypeDef*UartHandle)EOL{EOLEOL        RxLine++;                      //每接收到一个数据，进入回调数据长度加1EOL        DataBuff[RxLine-1]=RxBuff[0];  //把每次接收到的数据保存到缓存数组EOLEOLEOLEOL        if(DataBuff[RxLine-1]==0xff && DataBuff[RxLine-2]==0xff && DataBuff[RxLine-3]==0xff && DataBuff[RxL

標籤 : #json jobject asp.net c# c# 
詢問時間 : 2022-07-15
瀏覽次數 : 505
內文 : EOLdynamic jsonObject = JsonConvert.DeserializeObject(jsonTxt);EOLif (jsonObject.root.rtn != '1') 甚麼情況是1???EOLif (jsonObject.root.rtn != '0') 甚麼情況會是0EOLjsonObject.root.msgEOL.rootEOL.rtnEOL.msgEOL請問它們代表的是什麼QQEOL.root.rtn 輸出的0/1代表的是什麼?????EOL查了很久不知是不是關鍵字錯誤 沒有結果EOL


標題 : 如何挷定class去做項目區分
標籤 : css 網頁設計 菜鳥前端 javascript htm 
詢問時間 : 2022-07-15
瀏覽次數 : 994
內文 : EOL以下是我參考的codepen :https://codepen.io/mtbroomell/pen/yNwwdvEOL我表格上每樣產品都會有這個 +- 來控制增加減少數量，但他現在是以id 為單位，所以我有10樣產品就需要10個不同的ID  分別在html/js 都要有十個對應的ID code 這樣我覺得好沒效率~ 請問如何能使JS那邊挷我的 <span class='shop-item-title'>產品名稱</span>   去做對應的+-項目EOL以下為單一產品時的code:EOL<div class='shop-items'>EOL    <div class='shop-item'>EOL      <span class='shop-item-title'>炸排骨飯</span>EOL           <img class='shop-item-image' src='Images/item_1.jpg'>EOL               <div class='shop-item-details'>EOL                 <span class='shop-item-price'>價格: 80元</span>EOL                      <form><div class='